- https://medium.com/@yu4u/why-mobilenet-and-its-variants-e-g-shufflenet-are-fast-1c7048b9618d
- https://github.com/chullhwan-song/Reading-Paper/issues/44
- https://stackoverflow.com/questions/42786717/how-to-calculate-the-number-of-parameters-for-convolutional-neural-network

## Introduction

2018년도 Yusuke Uchida의 Meidum Article [Why MobileNet and Its Variants (e.g. ShuffleNet) Are Fast](https://medium.com/@yu4u/why-mobilenet-and-its-variants-e-g-shufflenet-are-fast-1c7048b9618d)과 각 논문들을 정리한 포스팅입니다. 이 포스팅은 efficient CNN모델들을 정리하고 왜 그들이 효율적인 방식을 취하는지에 대한 이유입니다. Yusuke Uchida는 시각화를 통하여 그들이 왜 효율적인 방식을 취하는지에 대해서 정리했습니다.

## Building Blocks Used in Efficient Models

시작부분에서는 우리 한번 다시 CNN모델의 block들을 만드는데 있어서 발생하는 Computational cost를 계산해 봅시다. Input image의 가로 세로가 <H,W>의 크기를 가지고, N개의 채널이 있는 이미지를, <K,K> 크기를 가지는 Convolution filter를 사용한다고 가정해 봅시다. 그리고 M은 이제 output channel이라고 생각해봅시다. 그렇다면 Computational Cost는 $HWNK^{2}M$ 이 될 것입니다.

<img src="./img/CNblock.png">

$$
\text{Computational Cost} = HW(\text{Output}), K^{2}(\text{Kernel Size}), NM(\text{Input and Output Channel})
$$

여기서 중요한 포인트는 일반적인 컨볼루션 연산의 Computational cost는 다음과 같은 요소들과 비례하게 됩니다. (1)Ouput feature map $HxW$, (2)Convolution Kernel size $K^{2}$, (3)Number of input and output channels $NxM$. 즉 Computational Cost의 경우에는 Spatital 그리고 Channel Domain에 영향을 받게 되는 것입니다.



## Convolution

이제 Convolution연산들을 input과 output의 관점에서 시각화를 해봅시다. Line의 갯수들은 input과 output의 spatial과 channel에서의 Computational Cost를 의미합니다.

<img src="./img/CNblock1.png" width=70%>

예를 들어, conv 3x3의 경우는 가장 흔하게 쓰는 Convolution 입니다. Input과 output이 local하게 connect 되어있는것에 반해서, channel의 경우에는 fully connected 되어 있습니다.

<img src="./img/CNblock2.png" width=70%>

그 다음, conv 1x1 를 살펴본다면, 이 모듈은 pointwise convolution으로 더 유명합니다. 주로 Channel size를 변화하기 위해서 사용하게 됩니다. Computation Cost는 $HWNM$으로 줄어듭니다 왜냐하면 K인 kernel이 1x1로 없어지게 됩니다. 그 결과 3x3과는 다르게 1/9로 computation cost가 감소하게 됩니다. 주로 Channel의 데이터를 섞는데 사용됩니다. 

In [37]:
def conv3x3(in_planes, out_planes, stride=1, groups=1,dilation=1):
    return nn.Conv2d(in_channel,out_channel,kernel_size=3, stride=1, 
              padding=1, groups=1, bias=False, dilation=dilation)

## Grouped Convolution
Grouped Convolution의 경우는 Channel을 기준으로 feature map을 묶게 됩니다. 그 결과 convolution들의 퍼포먼스는 각 그룹의 채널별로 independently 하게 됩니다.

만약에 G를 그룹이라고 한다면, Computational cost들은 $HWNK^{2}M/G$가 됩니다.

<img src="./img/CNblock3.png" width=70%>

Channel의 커넥션을 그룹별로 묶게 되면서 우리는 더 적은 computational cost를 확보할 수 있습니다.
<img src="./img/CNblock4.png" width=70%>

그리고 conv 1x1가 된다면, 이러한 경우를 ShuffleNet에서 구현하게 됩니다.

<img src="./img/CNblock5.png" width=70%>

conv 1x1에 Group을 3개로 나누게 된다면 점점더 많은 Computation을 제한 할 수 있습니다.

<img src="./img/CNblock6.png" width=70%>

## Depthwise Convolution

Depthwise convolution는 마치 각각의 input channel들이 independently하게 작동할 경우 사용됩니다. 이것은 input과 ouput 채널과 그룹의 갯수가 같을때와 동일합니다.

<img src="./img/CNblock7.png" width=70%>

Depthwise의 경우에는 computation cost를 Channel domain에서 줄일 수 있습니다.

## Channel Shuffle

Channel Shuffle의 경우에는 Channel의 순서를 바꾸는 연산을 의미합니다. 이 연산은 Tensor의 reshape와 transpose를 통해서 이루어집니다. 좀더 자세히 말하기 위해서 우리가 input channel을 GN'(Group x Number)라고 정의해 봅시다. 맨 처음에는 Transpose를 통해 (G,N')을 (N',G)로 만듭니다. 그리고 input과 동일한 shape으로 Group을 풀어줍니다. 마치 Flatten하는 것 처럼요. 여기서 Group이 의미하는 것은 함께 셔플될 채널들을 의미하게 됩니다.

<img src="./img/CNblock8.png" width=70%>

그룹 수에 대한 Shuffle입니다. Convolution의 기능은 하지 않지만 Channel이 셔플되는것을 확인 할 수 있습니다.

## Efficient Models

효율적인 모델을 짜기 위해서는, 적은 Computation cost를 얻기위해서 Spatial과 Channel도메인을 학습시키는 전략을 짜게 됩니다.

### ResNet(Bottleneck Version)
ResNet의 경우에는 conv1x1, conv3x3, conv1x1을 하나의 Bottleneck block으로 사용하게 됩니다. 첫번째 conv1x1은 input channel의 차원을 압축하게 됩니다. 이것은 conv3x3에 비해서 훨신 적은 computation cost를 가지게 됩니다. 그리고 conv1x1을 통해서 output channel을 회복하게 됩니다.
<img src="./img/CNblock10.png" width=70%>

### ResNeXt
ResNeXt 역시 효율적인 CNN모형입니다. 이것은 ResNet에서 conv3x3부분을 G=2인 Group Convolution 으로 대체하는 모형이 됩니다. Group을 만들기 위해서, conv1x1부분의 차원이 늘어나지만, 같은 Computational cost에 비해서 더 높은 성능을 보이게 됩니다.
<img src="./img/CNblock9.png" width=70%>

### MobileNet (Separable Conv)
